In [ ]:
import os
import math
import time
import inspect
from dataclasses import dataclass
import  torch
import torch.nn as nn
import torch.nn.functional as F


class CasualSelfAttention(nn.module):
    
    def __init__(self, config):
        super().__init__()
        assert config.n_embd % config.n_head == 0
        self.c_attn = nn.Linear(config.n_embd, 3 * config.n_embd)
        self.c_proj = nn.Liear(config.n_embd, config.n_embd)
        self.c_proj.NANOGPT_SCALE_INIT = 1
        self.n_head = config.n_head
        self.n_embd = config.n_embd
        
        
    def foward(self, x):
        B ,T, C = x.size() #batch size, sequence length, embedding dimensionality (n_embd)
        qkv = self.c_attn(x)
        q, k ,v = qkv.split(self.n_embd, dim=2)



class Block(nn.module):
    
    def __init__(self, config):
        super().__init__()
        self.ln_1 = nn.LayerNorm(config.n_embd)
        self.attn = CasualSelfAttention(config)
        self.ln_2 = nn.LayerNorm(config.n_embd)
        self.mlp = MLP(config)
        
    def forward(self, x):
        x = x + slef.attn(self.ln_1(x))
        x = x + self.mlp(self.ln_2(x))
        return x
    



@dataclass
class GPTconfig:
    block_size: int = 256
    vocab_size: int = 65
    n_layer: int = 6
    n_head: int = 6
    n_embd: int = 384
    

class GPT(nn.Module):
    
    def __init__(self, config):
        super().__init__()
        self.config = config
        
        self.transformer = nn.ModuleDict(dict(
            wte = nn.Embedding(config.vocab_size, config.n_embd),
            wpe = nn.Embedding(config.block_size, config.n_embd),
            h = nn.ModuleList([Block(config) for _ in range(config.n_layer)]),
            ln_f = nn.LayerNorm(config.n_embd)
        ))